## Import Libraries

In [1]:
import re
import json
import pandas as pd
import numpy as np
already_multiplied = False

## Extract Data

In [2]:
orders = pd.read_csv("data/orders_2015.csv", encoding="latin-1")
order_details = pd.read_csv("data/order_details_2015.csv", encoding="latin-1")
pizza_ingredients = pd.read_csv("data/pizza_types.csv", encoding="latin-1")

### Informe Calidad de Datos

In [3]:
informe = {"orders":{}, "order_details":{}, "pizza_types":{}}

#### orders.csv

In [4]:
informe["orders"]["desciption"] = "csv with every order made in 2015"
informe["orders"]["n_rows"] = orders.shape[0]
informe["orders"]["n_columns"] = orders.shape[1]
informe["orders"]["columns"] = {}
orders.head()

,order_id,date,time
0,1,01/01/2015,11:38:36
1,2,01/01/2015,11:57:40
2,3,01/01/2015,12:12:28
3,4,01/01/2015,12:16:31
4,5,01/01/2015,12:21:30


order_id

In [5]:
informe["orders"]["columns"]["order_id"] = {}
informe["orders"]["columns"]["order_id"]["description"] = "unique id for each order"
informe["orders"]["columns"]["order_id"]["n_nulls"] = orders["order_id"].isnull().sum()
informe["orders"]["columns"]["order_id"]["n_unique"] = orders["order_id"].nunique()

date

In [6]:
informe["orders"]["columns"]["date"] = {}
informe["orders"]["columns"]["date"]["description"] = "date of the order"
informe["orders"]["columns"]["date"]["n_nulls"] = orders["date"].isnull().sum()
informe["orders"]["columns"]["date"]["n_unique"] = orders["date"].nunique()

time

In [7]:
informe["orders"]["columns"]["time"] = {}
informe["orders"]["columns"]["time"]["description"] = "exact time of the order"
informe["orders"]["columns"]["time"]["n_nulls"] = orders["time"].isnull().sum()
informe["orders"]["columns"]["time"]["n_unique"] = orders["time"].nunique()

#### order_details.csv

In [8]:
informe["order_details"]["desciption"] = "csv with information about each order made in 2015"
informe["order_details"]["n_rows"] = order_details.shape[0]
informe["order_details"]["n_columns"] = order_details.shape[1]
informe["order_details"]["columns"] = {}
order_details.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


order_details_id

In [9]:
informe["order_details"]["columns"]["order_details_id"] = {}
informe["order_details"]["columns"]["order_details_id"]["description"] = "unique id for each pizza in each order"
informe["order_details"]["columns"]["order_details_id"]["n_nulls"] = order_details["order_details_id"].isnull().sum()
informe["order_details"]["columns"]["order_details_id"]["n_unique"] = order_details["order_details_id"].nunique()

order_id

In [10]:
informe["order_details"]["columns"]["order_id"] = {}
informe["order_details"]["columns"]["order_id"]["description"] = "unique id for each order"
informe["order_details"]["columns"]["order_id"]["n_nulls"] = order_details["order_id"].isnull().sum()
informe["order_details"]["columns"]["order_id"]["n_unique"] = order_details["order_id"].nunique()

pizza_id

In [11]:
informe["order_details"]["columns"]["pizza_id"] = {}
informe["order_details"]["columns"]["pizza_id"]["description"] = "unique id for each pizza and size"
informe["order_details"]["columns"]["pizza_id"]["n_nulls"] = order_details["pizza_id"].isnull().sum()
informe["order_details"]["columns"]["pizza_id"]["n_unique"] = order_details["pizza_id"].nunique()

quantity

In [12]:
informe["order_details"]["columns"]["quantity"] = {}
informe["order_details"]["columns"]["quantity"]["description"] = "number of pizzas ordered"
informe["order_details"]["columns"]["quantity"]["n_nulls"] = order_details["quantity"].isnull().sum()
informe["order_details"]["columns"]["quantity"]["n_unique"] = order_details["quantity"].nunique()

#### pizza_types.csv

In [13]:
informe["pizza_types"]["desciption"] = "csv with information about each pizza and its ingredients"
informe["pizza_types"]["n_rows"] = pizza_ingredients.shape[0]
informe["pizza_types"]["n_columns"] = pizza_ingredients.shape[1]
informe["pizza_types"]["columns"] = {}
pizza_ingredients.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


pizza_type_id

In [14]:
informe["pizza_types"]["columns"]["pizza_type_id"] = {}
informe["pizza_types"]["columns"]["pizza_type_id"]["description"] = "unique id for each pizza without size"
informe["pizza_types"]["columns"]["pizza_type_id"]["n_nulls"] = pizza_ingredients["pizza_type_id"].isnull().sum()
informe["pizza_types"]["columns"]["pizza_type_id"]["n_unique"] = pizza_ingredients["pizza_type_id"].nunique()

name

In [15]:
informe["pizza_types"]["columns"]["name"] = {}
informe["pizza_types"]["columns"]["name"]["description"] = "full name of the pizza"
informe["pizza_types"]["columns"]["name"]["n_nulls"] = pizza_ingredients["name"].isnull().sum()
informe["pizza_types"]["columns"]["name"]["n_unique"] = pizza_ingredients["name"].nunique()

category

In [16]:
informe["pizza_types"]["columns"]["category"] = {}
informe["pizza_types"]["columns"]["category"]["description"] = "category of the pizza"
informe["pizza_types"]["columns"]["category"]["n_nulls"] = pizza_ingredients["category"].isnull().sum()
informe["pizza_types"]["columns"]["category"]["n_unique"] = pizza_ingredients["category"].nunique()

ingredients

In [17]:
informe["pizza_types"]["columns"]["ingredients"] = {}
informe["pizza_types"]["columns"]["ingredients"]["description"] = "ingredients of the pizza"
informe["pizza_types"]["columns"]["ingredients"]["n_nulls"] = pizza_ingredients["ingredients"].isnull().sum()
informe["pizza_types"]["columns"]["ingredients"]["n_unique"] = pizza_ingredients["ingredients"].nunique()

### Save report in json

In [18]:
'''parse every int64 to python int'''
def recursive_int64_to_int(x):
    if isinstance(x, dict):
        for key, value in x.items():
            x[key] = recursive_int64_to_int(value)
    elif isinstance(x, np.int64) or isinstance(x, np.int32):
        x = int(x)
    return x

informe = recursive_int64_to_int(informe)
json.dump(informe, open("informe_calidad_datos.json", "w"), indent=4)

## Transform Data

- The task is to predict a ingredient distribution for a given day.  
- To achieve that, the best way to store the data would be as a dataframe with dates
as index and ingredients as columns,  
 so that each row is an ingredient distribution of that date, and can thus the dataframe can be directly used as the X data matrix in our model later on.

### Parse Types, Clean Data and Remove Irrelevant information

orders

In [19]:
'''Parse dates'''
date_format = "%d/%m/%Y"
orders["date"] = pd.to_datetime(orders["date"], format=date_format)
'''Drop time column'''
if "time" in orders.columns:
    orders.drop(columns=["time"], inplace=True)

order_details

In [20]:
'''Drop order_details_id column'''
if "order_details_id" in order_details.columns:
    order_details.drop(columns=["order_details_id"], inplace=True)

pizza_ingredients

In [21]:
'''Replace ï¿½Nduja with Nduja, found a couple cells bellow exploring each ingredient.'''
pizza_ingredients["ingredients"] = pizza_ingredients["ingredients"].str.replace("ï¿½Nduja", "Nduja")
'''Replace Artichoke with Artichokes, also found a couple cells bellow exploring each ingredient.'''
pizza_ingredients["ingredients"] = pizza_ingredients["ingredients"].str.replace("Artichokes", "Artichoke")
'''Drop name and category columns.'''
if "name" in pizza_ingredients.columns:
    pizza_ingredients.drop(columns=["name", "category"], inplace=True)
if "category" in pizza_ingredients.columns:
    pizza_ingredients.drop(columns=["category"], inplace=True)

### Build ingredients_per_day Dataframe

Split pizza id into name and size

In [22]:
if "pizza_id" in order_details.columns:
    size_pattern = re.compile(r"_([smlx]+)$")
    order_details["pizza"] = order_details["pizza_id"].apply(lambda x: re.sub(size_pattern, "", x))
    order_details["size"] = order_details["pizza_id"].apply(lambda x: re.search(size_pattern, x).group(1))
    order_details.drop(columns=["pizza_id"], inplace=True)

In [23]:
order_details.head(3)

,order_id,quantity,pizza,size
0,1,1,hawaiian,m
1,2,1,classic_dlx,m
2,2,1,five_cheese,l


Add dates

In [24]:
if not "date" in order_details.columns:
    order_details=order_details.merge(orders, on="order_id", how="left")

In [25]:
order_details.head(3)

,order_id,quantity,pizza,size,date
0,1,1,hawaiian,m,2015-01-01
1,2,1,classic_dlx,m,2015-01-01
2,2,1,five_cheese,l,2015-01-01


Add ingredients columns

In [26]:
'''Dictionary with each pizza and the ingredients it has.'''
pizza_ingredients_dict = {}
for index, row in pizza_ingredients.iterrows():
    pizza_ingredients_dict[row["pizza_type_id"]] = {}
    for ingredient in row["ingredients"].split(","):
        pizza_ingredients_dict[row["pizza_type_id"]][ingredient.strip()] = 1
pizza_ingredients_dict['bbq_ckn']

{'Barbecued Chicken': 1,
 'Red Peppers': 1,
 'Green Peppers': 1,
 'Tomatoes': 1,
 'Red Onions': 1,
 'Barbecue Sauce': 1}

In [27]:
'''Set with all the ingredients.'''
ingredients_set = set()
for pizza in pizza_ingredients_dict:
    ingredients_set.update(pizza_ingredients_dict[pizza])
sorted(list(ingredients_set))

['Alfredo Sauce',
 'Anchovies',
 'Artichoke',
 'Arugula',
 'Asiago Cheese',
 'Bacon',
 'Barbecue Sauce',
 'Barbecued Chicken',
 'Beef Chuck Roast',
 'Blue Cheese',
 'Brie Carre Cheese',
 'Calabrese Salami',
 'Capocollo',
 'Caramelized Onions',
 'Chicken',
 'Chipotle Sauce',
 'Chorizo Sausage',
 'Cilantro',
 'Coarse Sicilian Salami',
 'Corn',
 'Eggplant',
 'Feta Cheese',
 'Fontina Cheese',
 'Friggitello Peppers',
 'Garlic',
 'Genoa Salami',
 'Goat Cheese',
 'Gorgonzola Piccante Cheese',
 'Gouda Cheese',
 'Green Olives',
 'Green Peppers',
 'Italian Sausage',
 'Jalapeno Peppers',
 'Kalamata Olives',
 'Luganega Sausage',
 'Mozzarella Cheese',
 'Mushrooms',
 'Nduja Salami',
 'Onions',
 'Oregano',
 'Pancetta',
 'Parmigiano Reggiano Cheese',
 'Pears',
 'Peperoncini verdi',
 'Pepperoni',
 'Pesto Sauce',
 'Pineapple',
 'Plum Tomatoes',
 'Prosciutto',
 'Prosciutto di San Daniele',
 'Provolone Cheese',
 'Red Onions',
 'Red Peppers',
 'Ricotta Cheese',
 'Romano Cheese',
 'Sliced Ham',
 'Smoked Gou

Add a column for each ingredient

In [28]:
for ing in ingredients_set:
    order_details[ing] = order_details["pizza"].apply(lambda x: pizza_ingredients_dict[x].get(ing, 0))

In [29]:
order_details.head(3)

,order_id,quantity,pizza,size,date,Arugula,Genoa Salami,Coarse Sicilian Salami,Anchovies,Feta Cheese,...,Calabrese Salami,Asiago Cheese,Red Onions,Prosciutto,Gorgonzola Piccante Cheese,Spinach,Pesto Sauce,Bacon,Nduja Salami,Jalapeno Peppers
0,1,1,hawaiian,m,2015-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,classic_dlx,m,2015-01-01,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,2,1,five_cheese,l,2015-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Weight each ingredient by pizza size

In [30]:
size_weights = {"s": 1, "m": 2, "l": 3, "xl": 4, "xxl": 5}
order_details["num_size"] = order_details["size"].apply(lambda x: size_weights[x])

In [31]:
if not already_multiplied:
    for ing in ingredients_set:
        order_details[ing] = order_details[ing] * order_details["quantity"] * order_details["num_size"]
    already_multiplied = True
order_details.head(3)

,order_id,quantity,pizza,size,date,Arugula,Genoa Salami,Coarse Sicilian Salami,Anchovies,Feta Cheese,...,Asiago Cheese,Red Onions,Prosciutto,Gorgonzola Piccante Cheese,Spinach,Pesto Sauce,Bacon,Nduja Salami,Jalapeno Peppers,num_size
0,1,1,hawaiian,m,2015-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,2,1,classic_dlx,m,2015-01-01,0,0,0,0,0,...,0,2,0,0,0,0,2,0,0,2
2,2,1,five_cheese,l,2015-01-01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [32]:
'''Dataframe with ingredients for each day.'''
columns = ["date"] + list(ingredients_set)
ingredients_per_day = order_details[columns].groupby("date").sum()

In [33]:
'''Sort ingredient columns by alphabetical order.'''
ingredients_per_day = ingredients_per_day.reindex(sorted(ingredients_per_day.columns), axis=1)

In [34]:
ingredients_per_day.head(3)

,Alfredo Sauce,Anchovies,Artichoke,Arugula,Asiago Cheese,Bacon,Barbecue Sauce,Barbecued Chicken,Beef Chuck Roast,Blue Cheese,...,Romano Cheese,Sliced Ham,Smoked Gouda Cheese,Soppressata Salami,Spinach,Sun-dried Tomatoes,Thai Sweet Chilli Sauce,Thyme,Tomatoes,Zucchini
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,3,4,43,9,18,24,27,27,8,21,...,21,7,21,3,60,11,22,0,227,7
2015-01-02,2,14,69,9,24,19,19,19,14,18,...,18,21,18,12,61,7,19,0,220,21
2015-01-03,13,11,67,12,30,18,16,16,21,12,...,12,18,12,3,78,19,23,1,195,23


## Load

In [35]:
ingredients_per_day.to_csv("data/ingredients_per_day_2015.csv")